In [1]:
from faiss_module import make_db, make_fewshot_db, get_embedding
import pandas as pd

train_df = pd.read_csv('train.csv')
# fewshot_db = make_fewshot_db(train_df)
train_db = make_db(train_df,'./train_faiss_db')
train_retriever = train_db.as_retriever(search_type="similarity_score_threshold",
                search_kwargs={'score_threshold': 0.8,'k':3})

test_df = pd.read_csv('test.csv')
test_db = make_db(test_df,'./test_faiss_db')


Loading PDF files from: 16
Done. 1427 chunks
Creating FAISS DB


c:\ProgramData\anaconda3\envs\llm_project\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


Done.
Loading PDF files from: 9
Done. 153 chunks
Creating FAISS DB
Done.


In [2]:
import warnings

# 특정 경고 메시지를 무시하도록 설정
warnings.filterwarnings("ignore", message="No relevant docs were retrieved using the relevance score threshold")


rag_counter = {'0':0,'1':0,'2':0,'3':0,'4':0,'5':0}

for i , entry in enumerate(train_df.to_dict(orient='records')):
    # print(entry['Question'])
    # print(entry['Source_path'])
    train_retriever = train_db.as_retriever(search_type="similarity_score_threshold",
                search_kwargs={'score_threshold': 0.77,'k':2})
    rag = train_retriever.invoke(entry['Question'])

    num_rag = len(rag)
    if num_rag ==0 :
        print(i)
        print(entry['Question'])
        print(entry['Source_path'])
        rag_counter['0'] += 1
    if num_rag == 1:
        rag_counter['1'] += 1
    elif num_rag == 2:
        rag_counter['2'] += 1
    elif num_rag == 3:
        rag_counter['3'] += 1
    elif num_rag == 4:
        rag_counter['4'] += 1
    elif num_rag == 5:
        rag_counter['5'] += 1
print(rag_counter)

13
내부거래지출이란 무엇을 의미하며, 어떤 종류의 거래를 포함하고 있는가?
./train_source/1-1 2024 주요 재정통계 1권.pdf
14
보전지출이란 무엇을 의미하며, 어떤 상황에서 발생하는가?
./train_source/1-1 2024 주요 재정통계 1권.pdf
20
세출예산현액이란?
./train_source/1-1 2024 주요 재정통계 1권.pdf
35
2022년에는 어떤 분야에 민생경제 지원을 위해 가장 많은 예산이 배분되었는가?
./train_source/1-1 2024 주요 재정통계 1권.pdf
53
농어가 소득과 생활안전망 확충을 위해 어떤 지원이 추가되었는가?
./train_source/2024 나라살림 예산개요.pdf
55
이상기후로 인한 수해 관련 재해 대응을 위해 어떤 시설이 신규 설치되고 있으며, 그 목적은 무엇인가?
./train_source/2024 나라살림 예산개요.pdf
60
한국 정부가 녹색산업 육성 및 해외진출 지원을 강화하기 위해 취하고 있는 조치와 전기차 및 수소차에 대한 정책은 무엇인가?
./train_source/2024 나라살림 예산개요.pdf
65
장애인 연금이 어떻게 변화했는가?
./train_source/2024 나라살림 예산개요.pdf
67
원전산업 맞춤형 정책자금이 어떻게 운용되며, 어떤 목적으로 신설되었는가?
./train_source/2024 나라살림 예산개요.pdf
68
에너지 신산업 분야에서 어떤 분야의 인프라 투자가 확대되고 있는가?
./train_source/2024 나라살림 예산개요.pdf
71
한국 정부가 외국인투자 및 국내복귀 투자에 대한 인센티브를 강화하는 이유는 무엇인가?
./train_source/2024 나라살림 예산개요.pdf
72
청년농업인을 위해 어떤 방식으로 농지를 공공임대하고 있는가?
./train_source/2024 나라살림 예산개요.pdf
73
농업의 생산성을 높이고 노동 투입량을 줄이기 위해 어떤 방안을 추진하고 있는가?
./train_source/2024

In [3]:
rag_counter = {'0':0,'1':0,'2':0,'3':0,'4':0,'5':0,'6':0,'7':0,'8':0,'9':0}
test_retriver = test_db.as_retriever(search_type="similarity_score_threshold",
                search_kwargs={'score_threshold': 0.77,'k':2})
for i , entry in enumerate(test_df.to_dict(orient='records')):
    num_rag = len(test_retriver.invoke(entry['Question']))
    if num_rag ==0 :
        rag_counter['0'] += 1
    if num_rag == 1:
        rag_counter['1'] += 1
    elif num_rag == 2:
        rag_counter['2'] += 1
    elif num_rag == 3:
        rag_counter['3'] += 1
    elif num_rag == 4:
        rag_counter['4'] += 1
    elif num_rag == 5:
        rag_counter['5'] += 1
print(rag_counter)

{'0': 13, '1': 17, '2': 68, '3': 0, '4': 0, '5': 0, '6': 0, '7': 0, '8': 0, '9': 0}


In [12]:
i = 26
querry = test_df.iloc[i]['Question']
test_retriver = test_db.as_retriever(search_kwargs={'k': 3})
print("Q: ",querry)
len(test_retriver.invoke(querry))
# print(doc_list[0].page_content)
# print(doc_list[0].metadata)

Q:  하절기바우처와 동절기바우처의 2024년 예산 규모는 각각 얼마인가요?


3

In [13]:
test_retriver = test_db.as_retriever(search_kwargs={'search_tpye':'ngram','k': 3})
print(querry)
test_retriver.invoke(querry)

하절기바우처와 동절기바우처의 2024년 예산 규모는 각각 얼마인가요?


[Document(metadata={'source': './test_source/산업통상자원부_에너지바우처.pdf', 'page': 5}, page_content='(131.7 백만원 ), 전달체계 구축(34.6백만원 ), 주택관리공단 운영지원 (50백만원 )\n ㅇ 2023년도 예산 및 2024년도 예산 산출 세부내역 비교\n’23년 예산 ’24년 예산\n예산 산출내역 예산 산출내역\n 190,963○ 민간경상보조 (320-01 ) : 190,963 백만원\n가. 하절기바우처 (36,851 백만원 )\n ㆍ 85.7만 세대 × 4.3만원 = 36,851 백만원\n나. 동절기바우처 (129,656 백만원 )\n ㆍ 85.3만 세대 × 15.2만원 = 129,656 백만원\n다. 연탄쿠폰 (21,712 백만원 )\n ㆍ 4.6만 세대 × 47.2만원 = 21,712 백만원\n라. 등유바우처 (1,395 백만원 )\n ㆍ 0.45만 세대 × 31만원 = 1,395백만원\n마. 사업운영비 (1,349 백만원 )\n ㆍ 에너지복지 홍보: 328백만원\n ㆍ 시스템 고도화 : 520백만원\n ㆍ 콜센터 운영 : 280백만원\n ㆍ 패널조사 : 133.5백만원\n ㆍ 에너지바우처 전달체계 구축 : 37.5백만원\n ㆍ 주택관리공단 운영지원 : 50백만원685,606○ 민간경상보조 (320-01 ) : 685,606 백만원\n가. 하절기바우처 (60,950 백만원 )\n ㆍ 115.0만 세대 × 5.3만원 × 100% = 60,950 백만원\n나. 동절기바우처 (600,521 백만원 )\n ㆍ (’23확대) 113.1만 세대 × 30.4만원 × 70% = 240,677 백만원\n ㆍ (’24지원) 114.6만 세대 × 31.4만원 = 359,844 백만원\n다. 연탄쿠폰 (21,240 백만원 )\n ㆍ 4.5만 세대 × 47.2만원 = 21,240 백만원\n라. 등유바우처 (1,023 백만원 )\n ㆍ 0.33만 세대 × 31만원 = 1,023백만원\n마. 사업운영비 (1

In [6]:
test_retriver = test_db.as_retriever(search_type="similarity_score_threshold",
                search_kwargs={'score_threshold': 0.8})
print(querry)
test_retriver.invoke(querry)

2016년 재정성과관리제도의 환류 개선사항은 무엇인가?


[Document(metadata={'source': './test_source/「FIS 이슈&포커스」 22-2호 《재정성과관리제도》.pdf', 'page': 6}, page_content='-  성과관리 사각지대 해소, 사업성과평가의 예산편성 활용 확대를 위한 중점 추진과제를 제시\n-  평가부담 완화를 위한 개선과 함께 지출구조조정을 강화하는 등 사업성과평가 개편\n-  핵심사업평가를 위한 핵심 재정사업군을 선별하여 5년간 전주기 관리\n-  사업별 성과정보 DB 구축, 온라인 재정정보 공개의 사용자 친화성 개선, 재정성과포럼 운영 등 성과관리 인\n프라를확충\n‣ 이를바탕으로 정부는 「’23년 재정사업 성과관리 추진계획」을 12월까지 마련해 보고할 계획임    재정사업 자율평가의 전면 개편\n‣ 그간의재정성과관리제도 변화는 성과관리의 실효성 제고, 환경 변화에 대한 적응, 평가 및 피평가자 피로\n도경감,기계적인 예산과 성과관리 연동 폐지를 통한 재정사업 구조조정 합리화 등을 목적으로 이루어짐\n-  각부처의 목표 체계와 프로그램 예산체계는 2016년부터 일치하도록 하여 업무 중복을 경감\n-  평가체계를 단순화하여 피로도를 해소하고 단위사업에 대한 평가 및 환류 개선을 위한 노력은 2015년부터 \n지금까지계속되고 있음\n‣ 재정사업자율평가는 재정관리 합리성 제고를 위해 그동안 평가 대상 사업의 범위, 평가 주기, 상위 평가 \n방식등에서 지속적인 제도 개선이 이루어짐\n-  2015년까지는 기획재정부가 정한 공통 평가지침과 양식을 기본으로, 각 부처는 사업을 3년 주기로 점검하\n고,기획재정부는 메타 평가 방식으로 전수를 평가\n-  2016~2017년에는 연구개발사업을 포함시키고 평가 주기를 전 사업에 걸쳐 매년 평가하는 것으로 변경한 통\n합재정사업 평가제도 시행\n-  2018년에는 연구개발사업 평가를 다시 분리하고 기존의 재정사업 자율평가 제도로 환원하는 한편, 공통 평\n가지침과평가보고서 표준서식을 폐지하고 상위 평가 방식을 메타 평가 방식의

In [7]:
test_retriver = test_db.as_retriever(search_kwargs={'filter': {'source':test_df.iloc[i]['Source_path']}, 'score_threshold': 0.8, "k":3})
print(querry)
test_retriver.invoke(querry)

2016년 재정성과관리제도의 환류 개선사항은 무엇인가?


[Document(metadata={'source': './test_source/「FIS 이슈&포커스」 22-2호 《재정성과관리제도》.pdf', 'page': 6}, page_content='-  성과관리 사각지대 해소, 사업성과평가의 예산편성 활용 확대를 위한 중점 추진과제를 제시\n-  평가부담 완화를 위한 개선과 함께 지출구조조정을 강화하는 등 사업성과평가 개편\n-  핵심사업평가를 위한 핵심 재정사업군을 선별하여 5년간 전주기 관리\n-  사업별 성과정보 DB 구축, 온라인 재정정보 공개의 사용자 친화성 개선, 재정성과포럼 운영 등 성과관리 인\n프라를확충\n‣ 이를바탕으로 정부는 「’23년 재정사업 성과관리 추진계획」을 12월까지 마련해 보고할 계획임    재정사업 자율평가의 전면 개편\n‣ 그간의재정성과관리제도 변화는 성과관리의 실효성 제고, 환경 변화에 대한 적응, 평가 및 피평가자 피로\n도경감,기계적인 예산과 성과관리 연동 폐지를 통한 재정사업 구조조정 합리화 등을 목적으로 이루어짐\n-  각부처의 목표 체계와 프로그램 예산체계는 2016년부터 일치하도록 하여 업무 중복을 경감\n-  평가체계를 단순화하여 피로도를 해소하고 단위사업에 대한 평가 및 환류 개선을 위한 노력은 2015년부터 \n지금까지계속되고 있음\n‣ 재정사업자율평가는 재정관리 합리성 제고를 위해 그동안 평가 대상 사업의 범위, 평가 주기, 상위 평가 \n방식등에서 지속적인 제도 개선이 이루어짐\n-  2015년까지는 기획재정부가 정한 공통 평가지침과 양식을 기본으로, 각 부처는 사업을 3년 주기로 점검하\n고,기획재정부는 메타 평가 방식으로 전수를 평가\n-  2016~2017년에는 연구개발사업을 포함시키고 평가 주기를 전 사업에 걸쳐 매년 평가하는 것으로 변경한 통\n합재정사업 평가제도 시행\n-  2018년에는 연구개발사업 평가를 다시 분리하고 기존의 재정사업 자율평가 제도로 환원하는 한편, 공통 평\n가지침과평가보고서 표준서식을 폐지하고 상위 평가 방식을 메타 평가 방식의

In [8]:
test_db.similarity_search_with_relevance_scores(querry, k=10, score_threshold=0.8,filter={'source':test_df.iloc[i]['Source_path']})


[(Document(metadata={'source': './test_source/「FIS 이슈&포커스」 22-2호 《재정성과관리제도》.pdf', 'page': 6}, page_content='-  성과관리 사각지대 해소, 사업성과평가의 예산편성 활용 확대를 위한 중점 추진과제를 제시\n-  평가부담 완화를 위한 개선과 함께 지출구조조정을 강화하는 등 사업성과평가 개편\n-  핵심사업평가를 위한 핵심 재정사업군을 선별하여 5년간 전주기 관리\n-  사업별 성과정보 DB 구축, 온라인 재정정보 공개의 사용자 친화성 개선, 재정성과포럼 운영 등 성과관리 인\n프라를확충\n‣ 이를바탕으로 정부는 「’23년 재정사업 성과관리 추진계획」을 12월까지 마련해 보고할 계획임    재정사업 자율평가의 전면 개편\n‣ 그간의재정성과관리제도 변화는 성과관리의 실효성 제고, 환경 변화에 대한 적응, 평가 및 피평가자 피로\n도경감,기계적인 예산과 성과관리 연동 폐지를 통한 재정사업 구조조정 합리화 등을 목적으로 이루어짐\n-  각부처의 목표 체계와 프로그램 예산체계는 2016년부터 일치하도록 하여 업무 중복을 경감\n-  평가체계를 단순화하여 피로도를 해소하고 단위사업에 대한 평가 및 환류 개선을 위한 노력은 2015년부터 \n지금까지계속되고 있음\n‣ 재정사업자율평가는 재정관리 합리성 제고를 위해 그동안 평가 대상 사업의 범위, 평가 주기, 상위 평가 \n방식등에서 지속적인 제도 개선이 이루어짐\n-  2015년까지는 기획재정부가 정한 공통 평가지침과 양식을 기본으로, 각 부처는 사업을 3년 주기로 점검하\n고,기획재정부는 메타 평가 방식으로 전수를 평가\n-  2016~2017년에는 연구개발사업을 포함시키고 평가 주기를 전 사업에 걸쳐 매년 평가하는 것으로 변경한 통\n합재정사업 평가제도 시행\n-  2018년에는 연구개발사업 평가를 다시 분리하고 기존의 재정사업 자율평가 제도로 환원하는 한편, 공통 평\n가지침과평가보고서 표준서식을 폐지하고 상위 평가 방식을 메타 평가 방식

In [9]:
test_retriver = test_db.as_retriever(search_kwargs={'filter': {'source':test_df.iloc[i]['Source_path']}, "k":3})
print(querry)
test_retriver.invoke(querry)

2016년 재정성과관리제도의 환류 개선사항은 무엇인가?


[Document(metadata={'source': './test_source/「FIS 이슈&포커스」 22-2호 《재정성과관리제도》.pdf', 'page': 6}, page_content='-  성과관리 사각지대 해소, 사업성과평가의 예산편성 활용 확대를 위한 중점 추진과제를 제시\n-  평가부담 완화를 위한 개선과 함께 지출구조조정을 강화하는 등 사업성과평가 개편\n-  핵심사업평가를 위한 핵심 재정사업군을 선별하여 5년간 전주기 관리\n-  사업별 성과정보 DB 구축, 온라인 재정정보 공개의 사용자 친화성 개선, 재정성과포럼 운영 등 성과관리 인\n프라를확충\n‣ 이를바탕으로 정부는 「’23년 재정사업 성과관리 추진계획」을 12월까지 마련해 보고할 계획임    재정사업 자율평가의 전면 개편\n‣ 그간의재정성과관리제도 변화는 성과관리의 실효성 제고, 환경 변화에 대한 적응, 평가 및 피평가자 피로\n도경감,기계적인 예산과 성과관리 연동 폐지를 통한 재정사업 구조조정 합리화 등을 목적으로 이루어짐\n-  각부처의 목표 체계와 프로그램 예산체계는 2016년부터 일치하도록 하여 업무 중복을 경감\n-  평가체계를 단순화하여 피로도를 해소하고 단위사업에 대한 평가 및 환류 개선을 위한 노력은 2015년부터 \n지금까지계속되고 있음\n‣ 재정사업자율평가는 재정관리 합리성 제고를 위해 그동안 평가 대상 사업의 범위, 평가 주기, 상위 평가 \n방식등에서 지속적인 제도 개선이 이루어짐\n-  2015년까지는 기획재정부가 정한 공통 평가지침과 양식을 기본으로, 각 부처는 사업을 3년 주기로 점검하\n고,기획재정부는 메타 평가 방식으로 전수를 평가\n-  2016~2017년에는 연구개발사업을 포함시키고 평가 주기를 전 사업에 걸쳐 매년 평가하는 것으로 변경한 통\n합재정사업 평가제도 시행\n-  2018년에는 연구개발사업 평가를 다시 분리하고 기존의 재정사업 자율평가 제도로 환원하는 한편, 공통 평\n가지침과평가보고서 표준서식을 폐지하고 상위 평가 방식을 메타 평가 방식의

In [10]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import EmbeddingsFilter

embeddings = get_embedding()
embeddings_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.80)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=embeddings_filter, base_retriever=test_retriver
)
compression_retriever.invoke(querry)

[_DocumentWithState(metadata={'source': './test_source/「FIS 이슈&포커스」 22-2호 《재정성과관리제도》.pdf', 'page': 6}, page_content='-  성과관리 사각지대 해소, 사업성과평가의 예산편성 활용 확대를 위한 중점 추진과제를 제시\n-  평가부담 완화를 위한 개선과 함께 지출구조조정을 강화하는 등 사업성과평가 개편\n-  핵심사업평가를 위한 핵심 재정사업군을 선별하여 5년간 전주기 관리\n-  사업별 성과정보 DB 구축, 온라인 재정정보 공개의 사용자 친화성 개선, 재정성과포럼 운영 등 성과관리 인\n프라를확충\n‣ 이를바탕으로 정부는 「’23년 재정사업 성과관리 추진계획」을 12월까지 마련해 보고할 계획임    재정사업 자율평가의 전면 개편\n‣ 그간의재정성과관리제도 변화는 성과관리의 실효성 제고, 환경 변화에 대한 적응, 평가 및 피평가자 피로\n도경감,기계적인 예산과 성과관리 연동 폐지를 통한 재정사업 구조조정 합리화 등을 목적으로 이루어짐\n-  각부처의 목표 체계와 프로그램 예산체계는 2016년부터 일치하도록 하여 업무 중복을 경감\n-  평가체계를 단순화하여 피로도를 해소하고 단위사업에 대한 평가 및 환류 개선을 위한 노력은 2015년부터 \n지금까지계속되고 있음\n‣ 재정사업자율평가는 재정관리 합리성 제고를 위해 그동안 평가 대상 사업의 범위, 평가 주기, 상위 평가 \n방식등에서 지속적인 제도 개선이 이루어짐\n-  2015년까지는 기획재정부가 정한 공통 평가지침과 양식을 기본으로, 각 부처는 사업을 3년 주기로 점검하\n고,기획재정부는 메타 평가 방식으로 전수를 평가\n-  2016~2017년에는 연구개발사업을 포함시키고 평가 주기를 전 사업에 걸쳐 매년 평가하는 것으로 변경한 통\n합재정사업 평가제도 시행\n-  2018년에는 연구개발사업 평가를 다시 분리하고 기존의 재정사업 자율평가 제도로 환원하는 한편, 공통 평\n가지침과평가보고서 표준서식을 폐지하고 상위 평가 방식을